## Goal
* Gather sunset and sunrise times from an array of dates and locations and store results

https://sunrise-sunset.org/api

In [2]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

## Build Date Array

In [3]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range start at 6/30 to compensate for UTC
start_date = date(2017, 6, 30)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [4]:
print(dates[:4])
print(dates[-4:])

['2017-06-30', '2017-07-01', '2017-07-02', '2017-07-03']
['2017-10-28', '2017-10-29', '2017-10-30', '2017-10-31']


## Load Location Dictionary

In [5]:
# Load station location csv into a DataFrame
stations = pd.read_csv('../output/mesoweb-station_meta-180919.csv')

In [7]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 23 columns):
Unnamed: 0          12 non-null int64
COUNTRY             12 non-null object
COUNTY              12 non-null object
CWA                 12 non-null object
ELEVATION           12 non-null int64
ELEV_DEM            12 non-null float64
GACC                12 non-null object
ID                  12 non-null int64
LATITUDE            12 non-null float64
LONGITUDE           12 non-null float64
MNET_ID             12 non-null int64
NAME                12 non-null object
NWSFIREZONE         12 non-null object
NWSZONE             12 non-null object
PERIOD_OF_RECORD    12 non-null object
RESTRICTED          12 non-null bool
SGID                12 non-null object
SHORTNAME           12 non-null object
STATE               12 non-null object
STATUS              12 non-null object
STID                12 non-null object
TIMEZONE            12 non-null object
WIMS_ID             8 non-null float64
dty

## Build URLs

In [8]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['STID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date + '&formatted=0'
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [9]:
# Generate url list for api calls
url_batch = build_url_df(stations, dates)

In [14]:
print('Number of urls: ' + str(len(url_batch)))
print('----\n', url_batch[-1:])

Number of urls: 1488
----
 [{'station_id': 'KMSO', 'url': 'https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-10-31&formatted=0', 'date': '2017-10-31'}]


## Call API and store results

In [11]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json['results']

In [12]:
def save_results(df_day, filepath):
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            df_day.to_csv(f, header=False, index=False)
    else:
        df_day.to_csv(filepath, index=False)

In [16]:
%%time
successfull_calls = 0

# Loop over url_batch to call api and store results
for url in url_batch:
    # sleep to limit API call speed
    #time.sleep(3)
    
    print(url['station_id'] + '\n' + url['url'])
    result = api_call(url['url'])
    
    # include station_id
    result['station_id'] = url['station_id']
    
    # include date 
    result['date'] = url['date']
    
    df_day = pd.DataFrame.from_dict([result])

    save_results(df_day, '../output/sunrise_sunset-mesowest-utc.csv')
    
    successfull_calls += 1
    print('successfull_calls: ' + str(successfull_calls))

WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-06-30&formatted=0
successfull_calls: 1
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-01&formatted=0
successfull_calls: 2
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-02&formatted=0
successfull_calls: 3
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-03&formatted=0
successfull_calls: 4
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-04&formatted=0
successfull_calls: 5
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-05&formatted=0
successfull_calls: 6
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-06&formatted=0
successfull_calls: 7
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-07-07&f

successfull_calls: 64
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-02&formatted=0
successfull_calls: 65
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-03&formatted=0
successfull_calls: 66
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-04&formatted=0
successfull_calls: 67
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-05&formatted=0
successfull_calls: 68
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-06&formatted=0
successfull_calls: 69
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-07&formatted=0
successfull_calls: 70
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.25833300000001&date=2017-09-08&formatted=0
successfull_calls: 71
WSFM8
https://api.sunrise-sunset.org/json?lat=45.815556&lng=-114.258

successfull_calls: 127
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-03&formatted=0
successfull_calls: 128
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-04&formatted=0
successfull_calls: 129
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-05&formatted=0
successfull_calls: 130
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-06&formatted=0
successfull_calls: 131
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-07&formatted=0
successfull_calls: 132
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-08&formatted=0
successfull_calls: 133
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-07-09&formatted=0
successfull_calls: 134
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=

successfull_calls: 190
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-04&formatted=0
successfull_calls: 191
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-05&formatted=0
successfull_calls: 192
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-06&formatted=0
successfull_calls: 193
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-07&formatted=0
successfull_calls: 194
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-08&formatted=0
successfull_calls: 195
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-09&formatted=0
successfull_calls: 196
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=-113.95383100000001&date=2017-09-10&formatted=0
successfull_calls: 197
SUAM8
https://api.sunrise-sunset.org/json?lat=45.820728&lng=

successfull_calls: 253
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-05&formatted=0
successfull_calls: 254
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-06&formatted=0
successfull_calls: 255
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-07&formatted=0
successfull_calls: 256
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-08&formatted=0
successfull_calls: 257
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-09&formatted=0
successfull_calls: 258
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-10&formatted=0
successfull_calls: 259
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-11&formatted=0
successfull_calls: 260
TR285
https://api.sunrise-sunset.org/json?lat=45.982667&lng=-114.21625&date=2017-07-12&formatted=0
successfull_calls: 261
T

...

successfull_calls: 1152
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-05&formatted=0
successfull_calls: 1153
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-06&formatted=0
successfull_calls: 1154
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-07&formatted=0
successfull_calls: 1155
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-08&formatted=0
successfull_calls: 1156
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-09&formatted=0
successfull_calls: 1157
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-10&formatted=0
successfull_calls: 1158
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-11&formatted=0
successfull_calls: 1159
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-08-12&formatted=0
successf

successfull_calls: 1219
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-11&formatted=0
successfull_calls: 1220
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-12&formatted=0
successfull_calls: 1221
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-13&formatted=0
successfull_calls: 1222
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-14&formatted=0
successfull_calls: 1223
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-15&formatted=0
successfull_calls: 1224
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-16&formatted=0
successfull_calls: 1225
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-17&formatted=0
successfull_calls: 1226
MOMM8
https://api.sunrise-sunset.org/json?lat=46.718531&lng=-114.157311&date=2017-10-18&formatted=0
successf

successfull_calls: 1286
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-15&formatted=0
successfull_calls: 1287
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-16&formatted=0
successfull_calls: 1288
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-17&formatted=0
successfull_calls: 1289
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-18&formatted=0
successfull_calls: 1290
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-19&formatted=0
successfull_calls: 1291
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-20&formatted=0
successfull_calls: 1292
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-21&formatted=0
successfull_calls: 1293
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-08-22&formatted=0
successf

successfull_calls: 1353
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-21&formatted=0
successfull_calls: 1354
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-22&formatted=0
successfull_calls: 1355
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-23&formatted=0
successfull_calls: 1356
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-24&formatted=0
successfull_calls: 1357
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-25&formatted=0
successfull_calls: 1358
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-26&formatted=0
successfull_calls: 1359
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-27&formatted=0
successfull_calls: 1360
TS934
https://api.sunrise-sunset.org/json?lat=46.820725&lng=-114.100889&date=2017-10-28&formatted=0
successf

successfull_calls: 1421
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-26&formatted=0
successfull_calls: 1422
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-27&formatted=0
successfull_calls: 1423
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-28&formatted=0
successfull_calls: 1424
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-29&formatted=0
successfull_calls: 1425
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-30&formatted=0
successfull_calls: 1426
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-08-31&formatted=0
successfull_calls: 1427
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-09-01&formatted=0
successfull_calls: 1428
KMSO
https://api.sunrise-sunset.org/json?lat=46.92083&lng=-114.0925&date=2017-09-02&formatted=0
successfull_calls: 1429
KMSO
https://api

#### Additional Resources
[USNO](http://aa.usno.navy.mil/data/docs/RS_OneYear.php)